In [1]:
import time
from datetime import datetime
import numpy as np
import pandas as pd

In [11]:
actionDf2 = pd.read_csv('../data/JData_Action_201602.csv', dtype={'user_id': int, 'sku_id': int, 'time': str,
                                                                  'type': int, 'cate': int, 'brand': int})
actionDf2['model_id'] = actionDf2['model_id'].fillna(-1).astype(int)

很多重复值：

In [12]:
print len(actionDf2)
print len(actionDf2.drop_duplicates())

11485424
8729331


那些经常浏览京东的用户会是爬虫吗？

In [16]:
temp_df = actionDf2.groupby('user_id').count()
print temp_df.describe()
print np.percentile(temp_df['sku_id'], 99)

             sku_id          time      model_id          type          cate  \
count  73299.000000  73299.000000  73299.000000  73299.000000  73299.000000   
mean     156.692779    156.692779    156.692779    156.692779    156.692779   
std      301.489940    301.489940    301.489940    301.489940    301.489940   
min        1.000000      1.000000      1.000000      1.000000      1.000000   
25%       20.000000     20.000000     20.000000     20.000000     20.000000   
50%       61.000000     61.000000     61.000000     61.000000     61.000000   
75%      171.000000    171.000000    171.000000    171.000000    171.000000   
max    10993.000000  10993.000000  10993.000000  10993.000000  10993.000000   

              brand  
count  73299.000000  
mean     156.692779  
std      301.489940  
min        1.000000  
25%       20.000000  
50%       61.000000  
75%      171.000000  
max    10993.000000  
1340.0
[1 2 3 4 5 6]


看起来不是，还买了东西。

In [ ]:
hot_users = temp_df.index[temp_df['sku_id'] > 1340]
print np.unique(actionDf2['type'][actionDf2['user_id'] == hot_users[0]])

浏览量少的用户又在干嘛？

In [19]:
print np.percentile(temp_df['sku_id'], 1)

2.0


也有买东西。。。

In [20]:
cold_users = temp_df.index[temp_df['sku_id'] <= 2]
print np.unique(actionDf2['type'][actionDf2['user_id'] == hot_users[0]])

[1 2 3 4 5 6]


热门商品有哪些：

In [21]:
temp_df = actionDf2.groupby('sku_id').count()
print temp_df.describe()

            user_id          time      model_id          type          cate  \
count  20725.000000  20725.000000  20725.000000  20725.000000  20725.000000   
mean     554.182099    554.182099    554.182099    554.182099    554.182099   
std     2529.329370   2529.329370   2529.329370   2529.329370   2529.329370   
min        1.000000      1.000000      1.000000      1.000000      1.000000   
25%       10.000000     10.000000     10.000000     10.000000     10.000000   
50%       44.000000     44.000000     44.000000     44.000000     44.000000   
75%      198.000000    198.000000    198.000000    198.000000    198.000000   
max    97385.000000  97385.000000  97385.000000  97385.000000  97385.000000   

              brand  
count  20725.000000  
mean     554.182099  
std     2529.329370  
min        1.000000  
25%       10.000000  
50%       44.000000  
75%      198.000000  
max    97385.000000  


热门品类、品牌
cate = 8是最热的品类，正好也是需要预测的品类。品牌总数并不多，只有不到400个，估计全量的也只有500个。

In [23]:
temp_df = actionDf2.groupby('cate').count()
print temp_df
print temp_df.describe()
temp_df = actionDf2.groupby('brand').count()
print temp_df.describe()

      user_id   sku_id     time  model_id     type    brand
cate                                                       
4     2918671  2918671  2918671   2918671  2918671  2918671
5     1483361  1483361  1483361   1483361  1483361  1483361
6     1642371  1642371  1642371   1642371  1642371  1642371
7     1126849  1126849  1126849   1126849  1126849  1126849
8     3194275  3194275  3194275   3194275  3194275  3194275
9      888435   888435   888435    888435   888435   888435
10     211572   211572   211572    211572   211572   211572
11      19890    19890    19890     19890    19890    19890
            user_id        sku_id          time      model_id          type  \
count  8.000000e+00  8.000000e+00  8.000000e+00  8.000000e+00  8.000000e+00   
mean   1.435678e+06  1.435678e+06  1.435678e+06  1.435678e+06  1.435678e+06   
std    1.148581e+06  1.148581e+06  1.148581e+06  1.148581e+06  1.148581e+06   
min    1.989000e+04  1.989000e+04  1.989000e+04  1.989000e+04  1.989000e+04   
25%  

下面一段是用来取时间窗口的，顺便检查一下窗口的范围对不对：

In [3]:
print actionDf2.columns

mask1 = (actionDf2['time'] > '2016-02-01') & (actionDf2['time'] < '2016-02-02')
mask2 = (actionDf2['time'] > '2016-02-01') & (actionDf2['time'] < '2016-02-06')

print len(actionDf2['time'][mask1])
print np.unique(actionDf2['time'][mask1])

print len(actionDf2['time'][mask2])
l = np.unique(actionDf2['time'][mask2])
fun = lambda x: x[:10]
vfun = np.vectorize(fun)
print np.unique(vfun(l))

Index([u'user_id', u'sku_id', u'time', u'model_id', u'type', u'cate',
       u'brand'],
      dtype='object')
370712
['2016-02-01 00:00:00' '2016-02-01 00:00:01' '2016-02-01 00:00:03' ...,
 '2016-02-01 23:59:56' '2016-02-01 23:59:57' '2016-02-01 23:59:58']
1544497
['2016-02-01' '2016-02-02' '2016-02-03' '2016-02-04' '2016-02-05']


In [4]:
print actionDf2.head()
print len(np.unique(actionDf2['user_id']))

   user_id  sku_id                 time  model_id  type  cate  brand
0   266079  138778  2016-01-31 23:59:02       NaN     1     8    403
1   266079  138778  2016-01-31 23:59:03       0.0     6     8    403
2   200719   61226  2016-01-31 23:59:07       NaN     1     8     30
3   200719   61226  2016-01-31 23:59:08       0.0     6     8     30
4   263587   72348  2016-01-31 23:59:08       NaN     1     5    159
73299


In [17]:
userDf = pd.read_csv('../data/JData_User.csv')
userDf['age'].fillna('-1', inplace=True)  # *
userDf['sex'].fillna(2, inplace=True)
print userDf.head()
print len(np.unique(userDf['user_reg_tm']))

   user_id       age  sex  user_lv_cd user_reg_tm
0   200001  56������  2.0           5  2016-01-26
1   200002        -1  0.0           1  2016-01-26
2   200003   36-45��  1.0           4  2016-01-26
3   200004        -1  2.0           1  2016-01-26
4   200005   16-25��  0.0           4  2016-01-26
3456


In [24]:
productDf = pd.read_csv('../data/JData_Product.csv')
print productDf.head()
print np.unique(productDf['cate'])

actionProductDf = pd.merge(left=actionDf2, right=productDf, on='sku_id', how='right')

   sku_id  a1  a2  a3  cate  brand
0      10   3   1   1     8    489
1  100002   3   2   2     8    489
2  100003   1  -1  -1     8     30
3  100006   1   2   1     8    545
4   10001  -1   1   2     8    244
[8]


In [40]:
print actionProductDf.columns
print len(np.unique(actionProductDf['sku_id']))
print np.unique(actionProductDf['cate_x'].fillna('nan'))

Index([u'user_id', u'sku_id', u'time', u'model_id', u'type', u'cate_x',
       u'brand_x', u'a1', u'a2', u'a3', u'cate_y', u'brand_y'],
      dtype='object')
24187
[8]


所以，JData_Product.csv中的商品cate全为8与action数据没有冲突。

In [43]:
commentDf = pd.read_csv('../data/JData_Comment.csv')
print commentDf.head()
print np.unique(commentDf['dt'])
print len(np.unique(commentDf['sku_id']))
print np.unique(commentDf['bad_comment_rate'])

           dt  sku_id  comment_num  has_bad_comment  bad_comment_rate
0  2016-02-01    1000            3                1            0.0417
1  2016-02-01   10000            2                0            0.0000
2  2016-02-01  100011            4                1            0.0376
3  2016-02-01  100018            3                0            0.0000
4  2016-02-01  100020            3                0            0.0000
['2016-02-01' '2016-02-08' '2016-02-15' '2016-02-22' '2016-02-29'
 '2016-03-07' '2016-03-14' '2016-03-21' '2016-03-28' '2016-04-04'
 '2016-04-11' '2016-04-15']
46546
[  0.00000000e+00   6.00000000e-04   8.00000000e-04 ...,   8.00000000e-01
   8.75000000e-01   1.00000000e+00]


既然京东给的是每7天一次的comment数据，那时间窗口长度也就取7天吧。